In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
from tensorflow.keras.applications.resnet50 import preprocess_input


In [ ]:
modelos = [load_model('modelos/rn1.h5'),load_model('modelos/rn2.h5'),load_model('modelos/rn3.h5'),load_model('modelos/rn4.h5'),load_model('modelos/rn5.h5'),load_model('modelos/rn6.h5'),load_model('modelos/rn7.h5')]

In [ ]:
sample_submission_path = '/kaggle/input/histopathologic-cancer-detection/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)

nombres_imagenes = sample_submission['id'].values.tolist()
nombres_imagenes = [nombre + '.tif' for nombre in nombres_imagenes]

In [ ]:
predicciones_totales = []

ruta_imagenes = '/kaggle/input/histopathologic-cancer-detection/test'

imagenes = []
for nombre in nombres_imagenes:
    img_path = os.path.join(ruta_imagenes, nombre)
    if os.path.exists(img_path):  
        imagen = load_img(img_path, target_size=(96,96)) 
        imagen = img_to_array(imagen)               
        imagen = preprocess_input(imagen)         
                
        imagenes.append(imagen)

In [ ]:
imagenes=np.array(imagenes)

In [ ]:
for modelo in modelos:
    predicciones_modelo = modelo.predict(imagenes)
    predicciones_modelo_binarias = (predicciones_modelo > 0.5).astype(int)
    predicciones_totales.append(predicciones_modelo_binarias)

predicciones_totales = np.array(predicciones_totales)

etiquetas_finales = []

for i in range(predicciones_totales.shape[1]):  
    predicciones = predicciones_totales[:, i]  
    etiqueta_final = 1 if np.sum(predicciones) >= len(predicciones) / 2 else 0
    etiquetas_finales.append(etiqueta_final)



In [ ]:
df_resultados = pd.DataFrame({'id': nombres_imagenes, 'label': etiquetas_finales})
df_resultados['id'] = df_resultados['id'].str.replace('.tif', '', regex=False)
df_resultados.to_csv('resultados_predicciones_rnvgg.csv', index=False) #se genera el csv para enviar las predicciones a Kaggle